In [1]:
import importlib
import data_loader
importlib.reload(data_loader)

from data_loader import PointCloudLoader
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.3.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
loader = PointCloudLoader("hand_positions.txt")
loader.load_points()

slice_points = loader.filter_by_y_range(-0.1, 0.1)

In [ ]:


# Exemple avec des points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(slice_points)

# Ouvre une fenêtre native en dehors du notebook
o3d.visualization.draw_geometries([pcd], window_name="Coupe Y ∈ [-0.1, 0.1]")

In [2]:
loader = PointCloudLoader("hand_positions.txt")
loader.load_points()
all_points = loader.points


[OK] Loaded 10000000 points from hand_positions.txt


In [3]:
def create_line_set_from_point_cloud(pcd, distance_threshold=0.02):
    points = np.asarray(pcd.points)
    tree = KDTree(points)

    lines = []
    for i, point in enumerate(points):
        idx = tree.query_ball_point(point, r=distance_threshold)
        for j in idx:
            if i < j:
                lines.append([i, j])

    line_set = o3d.geometry.LineSet()
    line_set.points = pcd.points
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.paint_uniform_color([0.0, 0.0, 0.0])  # Noir
    return line_set

In [4]:
def create_wireframe_from_point_cloud(
    pcd: o3d.geometry.PointCloud,
    voxel_size: float = 0.5,
    k: int = 6
) -> o3d.geometry.LineSet:
    """
    Downsample un nuage de points et crée un LineSet en connectant chaque point à ses k plus proches voisins.

    Args:
        pcd (o3d.geometry.PointCloud): Le nuage de points d'origine.
        voxel_size (float): Taille du voxel pour le downsampling.
        k (int): Nombre de voisins à connecter.

    Returns:
        o3d.geometry.LineSet: Le maillage filaire.
    """
    # Downsample
    pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
    points = np.asarray(pcd_down.points)

    # KNN
    tree = KDTree(points)
    lines = []
    for i, point in enumerate(points):
        _, idxs = tree.query(point, k=k)
        for j in idxs:
            if i < j:
                lines.append([i, j])

    line_set = o3d.geometry.LineSet()
    line_set.points = pcd_down.points
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.paint_uniform_color([0, 0, 0])  # noir

    return line_set

In [6]:
def show_with_view(geometry, view="top"):
    """
    Affiche une géométrie avec une vue fixée (top, front, side).

    Args:
        geometry: o3d.geometry.PointCloud, TriangleMesh, etc.
        view: "top", "front", "side"
    """
    import open3d as o3d
    import numpy as np

    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=f"View: {view}", width=800, height=600)
    vis.add_geometry(geometry)

    # Vue selon orientation
    ctr = vis.get_view_control()
    ctr.set_zoom(0.8)

    if view == "top":       # Vue du dessus (plan X-Y, Z vertical)
        ctr.set_front([0, 0, -1])
        ctr.set_lookat([0, 0, 0])
        ctr.set_up([0, 1, 0])
    elif view == "front":   # Vue de face (plan X-Z, Y vertical)
        ctr.set_front([0, -1, 0])
        ctr.set_lookat([0, 0, 0])
        ctr.set_up([0, 0, 1])
    elif view == "side":    # Vue de côté (plan Y-Z, X vertical)
        ctr.set_front([-1, 0, 0])
        ctr.set_lookat([0, 0, 0])
        ctr.set_up([0, 0, 1])
    else:
        print(f"[WARN] Vue inconnue: {view}, utilisation par défaut")

    vis.run()
    vis.destroy_window()


In [8]:
# Ton nuage
pcd_all = o3d.geometry.PointCloud()
pcd_all.points = o3d.utility.Vector3dVector(all_points)

# Couleur par Z
z = np.asarray(all_points)[:, 2]
z_norm = (z - z.min()) / (z.max() - z.min())
colors = plt.cm.viridis(z_norm)[:, :3]
pcd_all.colors = o3d.utility.Vector3dVector(colors)

# Wireframe "léger" via downsample + KNN
wireframe = create_wireframe_from_point_cloud(pcd_all, voxel_size=0.01, k=6)

# Repère
frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)




In [ ]:
# Affichage
o3d.visualization.draw_geometries([pcd_all, wireframe, frame], window_name="Nuage avec wireframe optimisé")

In [11]:
show_with_view(pcd_all, view="bottom")    # vue de dessus (XY)
show_with_view(pcd_all, view="front")  # vue de face (XZ)
show_with_view(pcd_all, view="side")   # vue de côté (YZ)

# ou pour ton wireframe :
show_with_view(wireframe, view="top")


[WARN] Vue inconnue: bottom, utilisation par défaut


In [ ]:
# Ton nuage
pcd_all = o3d.geometry.PointCloud()
pcd_all.points = o3d.utility.Vector3dVector(all_points)

# Couleur (hauteur Z)
z = np.asarray(all_points)[:, 2]
z_norm = (z - z.min()) / (z.max() - z.min())
colors = plt.cm.cividis(z_norm)[:, :3]
pcd_all.colors = o3d.utility.Vector3dVector(colors)

# Génération des lignes entre points proches
lines = create_line_set_from_point_cloud(pcd_all, distance_threshold=0.02)

# Repère
frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)

# Affichage
o3d.visualization.draw_geometries([pcd_all, lines, frame], window_name="Nuage avec contours visuels")


In [ ]:
# Chargement des points (assumé que loader est déjà dispo)
all_points = loader.points

# Nuage complet
pcd_all = o3d.geometry.PointCloud()
pcd_all.points = o3d.utility.Vector3dVector(all_points)

# Couleur par hauteur (Z)
z = np.asarray(pcd_all.points)[:, 2]
z_norm = (z - z.min()) / (z.max() - z.min())
colors = plt.cm.viridis(z_norm)[:, :3]
pcd_all.colors = o3d.utility.Vector3dVector(colors)

# Nuage tranche
slice_points = loader.filter_by_y_range(-0.1, 0.1)
pcd_slice = o3d.geometry.PointCloud()
pcd_slice.points = o3d.utility.Vector3dVector(slice_points)
pcd_slice.paint_uniform_color([1.0, 0.0, 0.0])  # rouge

# Repère
frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)

# Reconstruction de surface non convexe via Poisson
pcd_all.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30)
)
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_all, depth=9)
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.0, 0.6, 1.0])  # bleu clair

# Affichage global : nuage, reconstruction et repère
o3d.visualization.draw_geometries(
    [pcd_all, pcd_slice, mesh, frame],
    window_name="Nuage + Tranche + Mesh Poisson"
)

In [ ]:
# Chargement des points (assumé que loader est déjà dispo)
all_points = loader.points

# Nuage complet
pcd_all = o3d.geometry.PointCloud()
pcd_all.points = o3d.utility.Vector3dVector(all_points)

# Estimation des normales
pcd_all.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30)
)

# Reconstruction Poisson
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd_all, depth=12
)
# Suppression des triangles avec une densité faible
densities = np.asarray(densities)
density_threshold = np.percentile(densities, 90)  # Garde les 90% des plus denses

# Masque des indices avec densité faible
indices = np.where(densities > density_threshold)[0]

# Filtrage du mesh en fonction des indices
mesh_filtered = mesh.select_by_index(indices)
mesh_filtered = mesh_filtered.filter_smooth_simple(number_of_iterations=10)

# Affichage du mesh filtré
o3d.visualization.draw_geometries([mesh_filtered], window_name="Filtered Poisson Mesh")

# # Calcul des normales pour un rendu propre
# mesh.compute_vertex_normals()

# # Couleur bleue claire pour bien voir
# mesh.paint_uniform_color([0.0, 0.6, 1.0])

# # Affichage du mesh seul
# o3d.visualization.draw_geometries([mesh], window_name="Poisson Mesh")

In [ ]:
def generate_and_save_poisson_mesh(points, output_path="poisson_mesh.ply", density_path="densities.npy", depth=9, voxel_size=0.01):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))

    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=depth)

    o3d.io.write_triangle_mesh(output_path, mesh)
    np.save(density_path, np.asarray(densities))

    print(f"[OK] Mesh saved to: {output_path}")
    print(f"[OK] Densities saved to: {density_path}")


In [ ]:
def load_and_filter_poisson_mesh(
    mesh_path="poisson_mesh.ply",
    density_path="densities.npy",
    keep_percentile=90,
    smooth_iter=10,
    colorize=True
):
    # Chargement du mesh
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    mesh.compute_vertex_normals()

    # Chargement des densités
    densities = np.load(density_path)
    threshold = np.percentile(densities, keep_percentile)
    indices = np.where(densities > threshold)[0]

    # Filtrage du mesh
    mesh_filtered = mesh.select_by_index(indices)
    mesh_filtered = mesh_filtered.filter_smooth_simple(number_of_iterations=smooth_iter)
    mesh_filtered.compute_vertex_normals()

    if colorize:
        # Normalisation des densités restantes
        filtered_densities = densities[indices]
        norm = (filtered_densities - filtered_densities.min()) / (filtered_densities.max() - filtered_densities.min())
        colors = plt.cm.plasma(norm)[:, :3]  # ou viridis, inferno, etc.
        mesh_filtered.vertex_colors = o3d.utility.Vector3dVector(colors)

    # Affichage
    o3d.visualization.draw_geometries([mesh_filtered], window_name="Filtered Poisson Mesh (colored)")


In [ ]:
generate_and_save_poisson_mesh(loader.points)

In [ ]:
load_and_filter_poisson_mesh("poisson_mesh.ply")

In [ ]:
load_and_filter_poisson_mesh(
    mesh_path="poisson_mesh.ply",
    density_path="densities.npy",
    keep_percentile=30,
    smooth_iter=5,
    colorize=True
)

In [ ]:
mesh = o3d.io.read_triangle_mesh("poisson_mesh.ply")
o3d.visualization.draw_geometries([mesh], window_name="Filtered Poisson Mesh")